# On MatplotLib

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import ConvexHull
import matplotlib.animation as animation
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from matplotlib.colors import LightSource

In [ ]:
def generate_points_data(num_points, scale):
    # Generate new random points scaled as specified
    return np.random.rand(num_points, 3) * scale
    
# Example: Generating datasets separately
num_frames = 100
num_points = 100
scale = 2

# For 1 object.
#data_points = generate_points_data(num_points, scale)
#datasets = [data_points for _ in range(num_frames)]

datasets = [generate_points_data(num_points, scale) for _ in range(num_frames)]

In [ ]:
class Faces():
    def __init__(self,tri, sig_dig=12, method="convexhull"):
        self.method=method
        self.tri = np.around(np.array(tri), sig_dig)
        self.grpinx = list(range(len(tri)))
        norms = np.around([self.norm(s) for s in self.tri], sig_dig)
        _, self.inv = np.unique(norms,return_inverse=True, axis=0)

    def norm(self,sq):
        cr = np.cross(sq[2]-sq[0],sq[1]-sq[0])
        return np.abs(cr/np.linalg.norm(cr))

    def isneighbor(self, tr1,tr2):
        a = np.concatenate((tr1,tr2), axis=0)
        return len(a) == len(np.unique(a, axis=0))+2

    def order(self, v):
        if len(v) <= 3:
            return v
        v = np.unique(v, axis=0)
        n = self.norm(v[:3])
        y = np.cross(n,v[1]-v[0])
        y = y/np.linalg.norm(y)
        c = np.dot(v, np.c_[v[1]-v[0],y])
        if self.method == "convexhull":
            h = ConvexHull(c)
            return v[h.vertices]
        else:
            mean = np.mean(c,axis=0)
            d = c-mean
            s = np.arctan2(d[:,0], d[:,1])
            return v[np.argsort(s)]

    def simplify(self):
        for i, tri1 in enumerate(self.tri):
            for j,tri2 in enumerate(self.tri):
                if j > i: 
                    if self.isneighbor(tri1,tri2) and \
                       self.inv[i]==self.inv[j]:
                        self.grpinx[j] = self.grpinx[i]
        groups = []
        for i in np.unique(self.grpinx):
            u = self.tri[self.grpinx == i]
            u = np.concatenate([d for d in u])
            u = self.order(u)
            groups.append(u)
        return groups



def calculate_camera_angles(num, rotate_axes, rotate_angle, camera_position):
    # Calculate base elev and azim from camera position
    x, y, z = camera_position
    r = math.sqrt(x**2 + y**2 + z**2)
    elev = math.degrees(math.asin(z / r))  # Base elevation angle
    azim = math.degrees(math.atan2(y, x))  # Base azimuthal angle

    # Apply rotation on top of base angles for each specified axis
    for axis in rotate_axes:
        if axis == 'x':
            elev += rotate_angle * num
        elif axis == 'y':
            azim += rotate_angle * num
        elif axis == 'z':
            elev += rotate_angle * num
            azim += rotate_angle * num

    return elev, azim



def plot_scatter(ax, points, alpha):
    # Create convex hull
    hull = ConvexHull(points)
    simplices = hull.simplices

    # Original triangles from the convex hull
    org_triangles = [points[s] for s in simplices]

    # Process faces
    f = Faces(org_triangles)
    g = f.simplify()


    # Create a light source
    ls = LightSource(azdeg=225.0, altdeg=45.0)
    
    # Assuming g is your list of faces for the Poly3DCollection
    facecolors = np.array(['green' for _ in g])
    edgecolors = np.array(['black' for _ in g])
    
    # Apply the light source to the face colors
    facecolors = ls.shade_faces(facecolors, g)
    
    # Plot the faces
    pc = Poly3DCollection(g, 
                          facecolors=facecolors, 
                          edgecolors=edgecolors,
                          linewidths=1.5, 
                          alpha=alpha, 
                          zorder=-3, 
                          shade=True
                         )
    ax.add_collection3d(pc)

    # Plot all points - non-vertices in blue
    for i in range(len(points)):
        if i not in hull.vertices:
            ax.scatter(points[i, 0], points[i, 1], points[i, 2], color='blue', s=10, zorder=3)

    # Plot vertices (corners) of the convex hull in red with larger size
    for vertex in hull.vertices:
        ax.scatter(points[vertex, 0], points[vertex, 1], points[vertex, 2], color='red', s=50, zorder=4)

    return ax


def update(num, ax, fig, datasets, rotate_axes, rotate_angle, camera_position, alpha):
    ax.clear()  # Clear current plot

    # Setting labels
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.set_title(f"Frame {num}")

    # Calculate camera angles and set camera view
    elev, azim = calculate_camera_angles(num, rotate_axes, rotate_angle, camera_position)
    ax.view_init(elev=elev, azim=azim)

    # Plot scatter points with transparency and return updated ax
    points = datasets[num]
    ax = plot_scatter(ax, points, alpha)

    return fig,

def run_animation(datasets, rotate_axes=['z'], rotate_angle=1, camera_position=(1, 1, 1), alpha=0.5, interval=100):
    # Create figure and axes
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Create animation with specified settings
    ani = animation.FuncAnimation(fig, update, frames=len(datasets), 
                                  fargs=(ax, fig, datasets, rotate_axes, rotate_angle, camera_position, alpha), 
                                  interval=interval)

    # Return the animation
    return ani

In [ ]:
# Example: Running the animation with the pre-generated datasets and specified transparency
ani = run_animation(datasets, rotate_axes=['x', 'y'], rotate_angle=3, camera_position=(1, 1, 1), alpha=1, interval=150)

In [ ]:
from IPython.display import HTML
HTML(ani.to_html5_video())

# MayAvi

In [1]:
from mayavi import mlab
from scipy.spatial import ConvexHull
import numpy as np
import math

def generate_points_data(num_points, scale):
    return np.random.rand(num_points, 3) * scale

class Faces():
    def __init__(self,tri, sig_dig=12, method="convexhull"):
        self.method=method
        self.tri = np.around(np.array(tri), sig_dig)
        self.grpinx = list(range(len(tri)))
        norms = np.around([self.norm(s) for s in self.tri], sig_dig)
        _, self.inv = np.unique(norms,return_inverse=True, axis=0)

    def norm(self,sq):
        cr = np.cross(sq[2]-sq[0],sq[1]-sq[0])
        return np.abs(cr/np.linalg.norm(cr))

    def isneighbor(self, tr1,tr2):
        a = np.concatenate((tr1,tr2), axis=0)
        return len(a) == len(np.unique(a, axis=0))+2

    def order(self, v):
        if len(v) <= 3:
            return v
        v = np.unique(v, axis=0)
        n = self.norm(v[:3])
        y = np.cross(n,v[1]-v[0])
        y = y/np.linalg.norm(y)
        c = np.dot(v, np.c_[v[1]-v[0],y])
        if self.method == "convexhull":
            h = ConvexHull(c)
            return v[h.vertices]
        else:
            mean = np.mean(c,axis=0)
            d = c-mean
            s = np.arctan2(d[:,0], d[:,1])
            return v[np.argsort(s)]

    def simplify(self):
        for i, tri1 in enumerate(self.tri):
            for j,tri2 in enumerate(self.tri):
                if j > i: 
                    if self.isneighbor(tri1,tri2) and \
                       self.inv[i]==self.inv[j]:
                        self.grpinx[j] = self.grpinx[i]
        groups = []
        for i in np.unique(self.grpinx):
            u = self.tri[self.grpinx == i]
            u = np.concatenate([d for d in u])
            u = self.order(u)
            groups.append(u)
        return groups

def calculate_camera_angles(num, rotate_axes, rotate_angle, camera_position):
    # Calculate base elev and azim from camera position
    x, y, z = camera_position
    r = math.sqrt(x**2 + y**2 + z**2)
    elev = math.degrees(math.asin(z / r))  # Base elevation angle
    azim = math.degrees(math.atan2(y, x))  # Base azimuthal angle

    # Apply rotation on top of base angles for each specified axis
    for axis in rotate_axes:
        if axis == 'x':
            elev += rotate_angle * num
        elif axis == 'y':
            azim += rotate_angle * num
        elif axis == 'z':
            elev += rotate_angle * num
            azim += rotate_angle * num

    return elev, azim

def plot_scatter(points, num, alpha):
    hull = ConvexHull(points)
    simplices = hull.simplices
    org_triangles = [points[s] for s in simplices]
    f = Faces(org_triangles)
    g = f.simplify()
    
    # Plot all points
    mlab.points3d(points[:, 0], points[:, 1], points[:, 2], color=(0, 0, 1), scale_factor=0.05)

    # Plot vertices of the convex hull
    mlab.points3d(hull.points[hull.vertices, 0], 
                  hull.points[hull.vertices, 1], 
                  hull.points[hull.vertices, 2], 
                  color=(1, 0, 0), 
                  scale_factor=0.1)

    # Plot the faces of the convex hull with adjusted opacity
    for face in g:
        mlab.triangular_mesh([v[0] for v in face], 
                             [v[1] for v in face], 
                             [v[2] for v in face], 
                             [[0, 1, 2]], 
                             color=(0, 1, 0), 
                             opacity=alpha)

    # Initialize text, positioned at the top with larger width
    #time_text = mlab.title('sss', size=32)
    #time_text.x_position = 0.5
    #time_text.y_position = 0.95
    # Update text
    
    time_text = mlab.text(0.01, 0.9, '', width=0.15)  # Increased width for larger text

    time_text.text = f'Time: {num}'


@mlab.animate(delay=100)
def anim(datasets, rotate_axes, rotate_angle, camera_position):
    f = mlab.gcf()
    text = mlab.text(0.01, 0.01, '', width=0.1)  # Initialize text, position it at bottom left

    while True:
        for num, points in enumerate(datasets):
            mlab.clf()
            plot_scatter(points, num, alpha=0.5)
            mlab.view(*calculate_camera_angles(num, rotate_axes, rotate_angle, camera_position))
            f.scene.render()
            yield

# Generate datasets
num_frames = 100
num_points = 100
scale = 2
datasets = [generate_points_data(num_points, scale) for _ in range(num_frames)]

# Run animation
anim(datasets, ['x', 'y'], 3, (1, 1, 1))

mlab.show()



No data: cannot use a Filter/Module/ModuleManager.


f
g
[array([[0.1206189 , 1.57820599, 1.81817969],
       [0.58997798, 1.84838755, 1.99982911],
       [0.00903122, 1.8422539 , 1.03909411]]), array([[1.41053357, 0.01859244, 1.23013698],
       [1.40993764, 0.22761376, 1.93213178],
       [1.98405054, 0.06580716, 0.01153062]]), array([[0.67929131, 1.97046617, 0.12675707],
       [0.71731384, 0.19870738, 0.02011854],
       [0.11459125, 1.56985398, 0.29376293]]), array([[0.17959041, 0.02031017, 1.33211929],
       [1.41053357, 0.01859244, 1.23013698],
       [1.40993764, 0.22761376, 1.93213178]]), array([[1.79590406e-01, 2.03101672e-02, 1.33211929e+00],
       [1.58250695e-01, 5.03232065e-04, 7.27177269e-01],
       [9.03121638e-03, 1.84225390e+00, 1.03909411e+00]]), array([[0.17959041, 0.02031017, 1.33211929],
       [0.1206189 , 1.57820599, 1.81817969],
       [0.00903122, 1.8422539 , 1.03909411]]), array([[0.80612487, 0.32039037, 1.84948523],
       [1.40993764, 0.22761376, 1.93213178],
       [0.58997798, 1.84838755, 1.99982911]]), 

: 

In [2]:
from scipy.spatial import ConvexHull
import numpy as np
import json

def generate_points_data(num_points, scale):
    return np.random.rand(num_points, 3) * scale

class Faces:
    def __init__(self, tri, sig_dig=12, method="convexhull"):
        self.method = method
        self.tri = np.around(np.array(tri), sig_dig)
        self.grpinx = list(range(len(tri)))
        norms = np.around([self.norm(s) for s in self.tri], sig_dig)
        _, self.inv = np.unique(norms, return_inverse=True, axis=0)

    def norm(self, sq):
        cr = np.cross(sq[2] - sq[0], sq[1] - sq[0])
        return np.abs(cr / np.linalg.norm(cr))

    def isneighbor(self, tr1, tr2):
        a = np.concatenate((tr1, tr2), axis=0)
        return len(a) == len(np.unique(a, axis=0)) + 2

    def order(self, v):
        if len(v) <= 3:
            return v
        v = np.unique(v, axis=0)
        n = self.norm(v[:3])
        y = np.cross(n, v[1] - v[0])
        y = y / np.linalg.norm(y)
        c = np.dot(v, np.c_[v[1] - v[0], y])
        if self.method == "convexhull":
            h = ConvexHull(c)
            return v[h.vertices]
        else:
            mean = np.mean(c, axis=0)
            d = c - mean
            s = np.arctan2(d[:, 0], d[:, 1])
            return v[np.argsort(s)]

    def simplify(self):
        for i, tri1 in enumerate(self.tri):
            for j, tri2 in enumerate(self.tri):
                if j > i:
                    if self.isneighbor(tri1, tri2) and self.inv[i] == self.inv[j]:
                        self.grpinx[j] = self.grpinx[i]
        groups = []
        for i in np.unique(self.grpinx):
            u = self.tri[self.grpinx == i]
            u = np.concatenate([d for d in u])
            u = self.order(u)
            groups.append(u)
        return groups

def calculate_data_for_visualization(points):
    hull = ConvexHull(points)
    simplices = hull.simplices
    org_triangles = [points[s] for s in simplices]

    f = Faces(org_triangles)
    faces_simplified = f.simplify()

    # Prepare JSON data
    data = {
        "blue_points": points.tolist(),
        "red_points": hull.points[hull.vertices].tolist(),
        "faces": [face.tolist() for face in faces_simplified]
    }
    return json.dumps(data, indent=4)

# Example usage
num_points = 100
scale = 2

points = generate_points_data(num_points, scale)

json_data = calculate_data_for_visualization(points)
print(json_data)

{
    "blue_points": [
        [
            0.5513964607406667,
            0.3511147016362972,
            1.3015995961109428
        ],
        [
            0.47105439521998327,
            1.3879688525519156,
            1.3993341094311806
        ],
        [
            1.6880312898852627,
            0.2471415215263777,
            0.7243964455352874
        ],
        [
            0.8819601019602452,
            0.43807578153565085,
            0.11570564619556345
        ],
        [
            1.6271520165951894,
            0.6707707445748861,
            0.5167308479373867
        ],
        [
            1.7473986413502458,
            1.609080478028415,
            0.27677255877462015
        ],
        [
            1.3509032062654784,
            0.6353597254114562,
            0.4346998299810436
        ],
        [
            0.7215029286599006,
            1.1620027626083096,
            1.1365427524653215
        ],
        [
            1.1313224716831451,
    